In [1]:
# Importando as Bibliotecas
import pandas as pd
from sklearn.svm import OneClassSVM
#from sklearn.model_selection import train_test_split

In [2]:
# Lendo os arquivos
csv_populacao_tratada = pd.read_csv('dataframe_populacao_tratada.csv')
csv_portfolio1 = pd.read_csv('portfolio1.csv')
csv_portfolio2 = pd.read_csv('portfolio2.csv')
csv_portfolio3 = pd.read_csv('portfolio3.csv')

# Criando DataFrame para cada um dos arquivos
df_populacao = pd.DataFrame(csv_populacao_tratada)
df_portfolio1 = pd.DataFrame(csv_portfolio1)
df_portfolio2 = pd.DataFrame(csv_portfolio2)
df_portfolio3 = pd.DataFrame(csv_portfolio3)

In [3]:
df_populacao.shape

(462298, 267)

In [42]:
df_populacao.dtypes.value_counts()

int64      263
float64      3
object       1
dtype: int64

In [5]:
df_portfolio1.shape

(555, 182)

In [59]:
df_portfolio2.shape

(566, 2)

In [60]:
df_portfolio2.columns

Index(['Unnamed: 0', 'id'], dtype='object')

In [7]:
df_portfolio3.shape

(265, 2)

## TREINO e TESTE

In [8]:
# DataFrame (df_entrada): é o DataFrame criado a partir da lista de clientes do usuário (upload arquivo)
# Queremos apenas o valor de "id". As outras colunas "não interessam, pois já temos as informações na população
df_entrada = pd.DataFrame(df_portfolio1['id'])

# Criando a coluna "target" no DataFrame que recebemos de entrada (que contém a lista das empresas (portfolio))
# E atribuindo o valor target = 1
df_entrada['target'] = 1

In [9]:
df_entrada.head()

,id,target
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,1
1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...,1
2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...,1
3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...,1
4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...,1


In [10]:
df_entrada.columns

Index(['id', 'target'], dtype='object')

In [11]:
# Verificando o tamanho do DataFrame com as informacoes do ID do portfólio de entrada do usuário
df_entrada.shape

(555, 2)

In [12]:
df_completo = pd.merge(df_populacao, df_entrada, how='outer', on ='id')

In [13]:
df_completo.shape

(462298, 268)

In [14]:
# Verificando se a quantidade de linhas com "target=1" é a mesma do (df_entrada)
df_completo[df_completo['target'] == 1].shape

(555, 268)

In [15]:
if ('Unnamed: 0'in df_completo.columns):
    df_completo.drop(columns = 'Unnamed: 0', inplace = True)

In [16]:
df_completo.shape

(462298, 267)

In [17]:
# Colocando zeros onde "Target" é NaN (ou seja, != 1)
df_completo['target'].fillna(value = 0, inplace=True)

In [18]:
df_completo['target'].isna().sum()

0

In [19]:
# Vamos trabalhar com o DataFrame (df_completo)

### Definindo: TREINO e TESTE

In [20]:
# Salvando os valores de "id" em (ArmazenandoID)
#df_ArmazenandoID = df_completo['id']
#df_completo.drop(columns = 'id', inplace=True)

In [21]:
# Definindo a base de TREINO (portfolio): Y_treino, X_treino
# Salvando em "df_completo_TREINO" as linhas do DataFrame que tem "target = 1"
df_completo_TREINO = df_completo.loc[(df_completo['target'] == 1)]
Y_treino = df_completo_TREINO['target']
X_treino = df_completo_TREINO.drop(columns ='target')

In [22]:
# Definindo a base de TEST: X_teste, Y_predito
# (população, excluindo as empresas que fazem parte da lista da empresa, ou seja, "target = 0")
# Salvando em "df_result_0" as linhas do DataFrame que tem "target = 0"
df_completo_TESTE = df_completo.loc[(df_completo['target'] == 0)]
X_teste = df_completo_TESTE.drop(columns ='target')
# Y_predito: vai ser o resultado da previsão do Modelo

In [23]:
#df_resultado_PARCIAL_treino = X_treino

In [24]:
lista_ID_teste = list(X_teste['id'])
lista_ID_treino = list(X_treino['id'])

In [25]:
df_ID_teste = pd.DataFrame({'id': lista_ID_teste})

In [26]:
X_teste.drop(columns = 'id', inplace = True)
X_treino.drop(columns = 'id', inplace = True)

In [27]:
X_teste.shape

(461743, 265)

### SVM.OneClass

In [28]:
from sklearn.svm import OneClassSVM

In [29]:
one_class_SVM = OneClassSVM(kernel='rbf')
resultado_OneClass_FIT = one_class_SVM.fit(X_treino)
resultado_OneClass_PREDICT = one_class_SVM.predict(X_teste)

In [30]:
#resultado_OneClass_FIT

In [31]:
#resultado_OneClass_PREDICT

In [32]:
#df_resultado = pd.DataFrame(df_completo)

In [33]:
df_resultado = df_ID_teste

In [34]:
df_resultado_OneClass_PREDICT = pd.DataFrame({'resultado_OneClass_PREDICT': resultado_OneClass_PREDICT})

In [57]:
df_resultado_OneClass_PREDICT['resultado_OneClass_PREDICT'].unique()

array([-1,  1])

In [36]:
df_resultado = df_resultado.join(df_resultado_OneClass_PREDICT)

In [37]:
df_auxiliar_NAO = df_resultado[df_resultado['resultado_OneClass_PREDICT'] == -1]

In [38]:
df_auxiliar_NAO.shape

(444482, 2)

In [39]:
df_auxiliar_SIM = df_resultado[df_resultado['resultado_OneClass_PREDICT'] == 1]

In [40]:
df_auxiliar_SIM.shape

(17261, 2)

In [41]:
df_recomendar = df_auxiliar_SIM['id']